# Install and Import

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
import time
from datetime import datetime, timezone
import torch
import torchvision.transforms as transforms
from PIL import Image
import io
import os
import firebase_admin # Import Firebase Admin SDK
from firebase_admin import credentials
from firebase_admin import firestore # Import Cloud Firestore

import concurrent.futures
from ultralytics import YOLO
import collections

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Connect DataBase And APP

In [4]:


def initialize_firebase():
    global db
    try:
        # Kiểm tra xem app Firebase đã được khởi tạo chưa
        if not firebase_admin._apps:
            # Sử dụng thông tin xác thực từ file JSON
            cred = credentials.Certificate("/content/drive/MyDrive/Project/Dự án cntt 2 /dataset/density-traffic-app-firebase-adminsdk-fbsvc-0f73d667a9.json")
            # Khởi tạo Firebase Admin SDK
            firebase_admin.initialize_app(cred)
            print("Firebase Admin SDK initialized successfully.")

        # Lấy đối tượng Firestore client
        db = firestore.client()
        print("Firestore client obtained.")
        return True
    except FileNotFoundError:
        print(f"ERROR: Firebase service account key file not found at {cred}")
        return False
    except Exception as e:
        print(f"ERROR initializing Firebase: {e}")
        return False

In [5]:
MODEL_PATH = "/content/drive/MyDrive/Project/Dự án cntt 2 /dataset/YOLO8m_v1_0001.pt"


def load_model():
    global model
    if not os.path.exists(MODEL_PATH):
        print(f"ERROR: Model file not found at {MODEL_PATH}")
        # Bạn có thể muốn dừng ứng dụng hoặc xử lý lỗi khác ở đây
        return False
    try:
        # Tải model từ file .pt đã script/trace
        model = YOLO(MODEL_PATH)
        model.eval() # Chuyển model sang chế độ đánh giá
        print(f"Model PyTorch loaded successfully from {MODEL_PATH}")
        return True
    except Exception as e:
        print(f"ERROR loading PyTorch model: {e}")
        return False

# Input Data

In [6]:
import requests
import time
import os
from urllib.parse import urlparse, parse_qs

# Danh sách URL của 20 camera
camera_page_urls = [
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=56de42f611f398ec0c481291&camLocation=V%C3%B5%20V%C4%83n%20Ki%E1%BB%87t%20-%20Nguy%E1%BB%85n%20Tri%20Ph%C6%B0%C6%A1ng%201&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=56de42f611f398ec0c481297&camLocation=V%C3%B5%20V%C4%83n%20Ki%E1%BB%87t%20-%20Nguy%E1%BB%85n%20Tri%20Ph%C6%B0%C6%A1ng%202&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=56de42f611f398ec0c481293&camLocation=V%C3%B5%20V%C4%83n%20Ki%E1%BB%87t%20-%20H%E1%BA%A3i%20Th%C6%B0%E1%BB%A3ng%20L%C3%A3n%20%C3%94ng%201&camMode=camera&videoUrl=http://camera.thongtingiaothong.vn/s/56de42f611f398ec0c481293/index.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5b632a79fd4edb0019c7dc0f&camLocation=Nguy%E1%BB%85n%20Tri%20Ph%C6%B0%C6%A1ng%20-%20Tr%E1%BA%A7n%20H%C6%B0ng%20%C4%90%E1%BA%A1o&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=662b4efc1afb9c00172d86bc&camLocation=Tr%E1%BA%A7n%20H%C6%B0ng%20%C4%90%E1%BA%A1o%20-%20Tr%E1%BA%A7n%20Ph%C3%BA&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5d8cd1f9766c880017188938&camLocation=Nguy%E1%BB%85n%20Tri%20Ph%C6%B0%C6%A1ng%20-%20Tr%E1%BA%A7n%20Ph%C3%BA&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5d8cd49f766c880017188944&camLocation=Nguy%E1%BB%85n%20Tri%20Ph%C6%B0%C6%A1ng%20-%20Nguy%E1%BB%85n%20Tr%C3%A3i&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=66b1c190779f740018673ed4&camLocation=Nguy%E1%BB%85n%20Tr%C3%A3i%20-%20Tr%E1%BA%A7n%20Ph%C3%BA&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5b632b60fd4edb0019c7dc12&camLocation=H%E1%BB%93ng%20B%C3%A0ng%20-%20Ng%C3%B4%20Quy%E1%BB%81n%201&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5deb576d1dc17d7c5515ad20&camLocation=H%E1%BB%93ng%20B%C3%A0ng%20-%20Ng%C3%B4%20Quy%E1%BB%81n%202&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=63b3c274bfd3d90017e9ab93&camLocation=H%E1%BB%93ng%20B%C3%A0ng%20-%20Ph%C3%B9%20%C4%90%E1%BB%95ng%20Thi%C3%AAn%20V%C6%B0%C6%A1ng&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5b728aafca0577001163ff7e&camLocation=H%E1%BB%93ng%20B%C3%A0ng%20-%20Ch%C3%A2u%20V%C4%83n%20Li%C3%AAm&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=662b4e201afb9c00172d85f9&camLocation=H%E1%BB%93ng%20B%C3%A0ng%20-%20T%E1%BA%A1%20Uy%C3%AAn&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5deb576d1dc17d7c5515ad21&camLocation=N%C3%BAt%20giao%20Ng%C3%A3%20s%C3%A1u%20Nguy%E1%BB%85n%20Tri%20Ph%C6%B0%C6%A1ng&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=649da419a6068200171a6c90&camLocation=Nguy%E1%BB%85n%20Ch%C3%AD%20Thanh%20-%20Ng%C3%B4%20Quy%E1%BB%81n&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=66f126e8538c780017c9362f&camLocation=Nguy%E1%BB%85n%20Ch%C3%AD%20Thanh%20-%20Nguy%E1%BB%85n%20Kim&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=662b4e8e1afb9c00172d865c&camLocation=L%C3%BD%20Th%C6%B0%E1%BB%9Dng%20Ki%E1%BB%87t%20-%20Nguy%E1%BB%85n%20Ch%C3%AD%20Thanh&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=662b4ecb1afb9c00172d8692&camLocation=Nguy%E1%BB%85n%20Ch%C3%AD%20Thanh%20-%20Thu%E1%BA%ADn%20Ki%E1%BB%81u&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=5deb576d1dc17d7c5515ad1f&camLocation=H%C3%B9ng%20V%C6%B0%C6%A1ng%20-%20Ng%C3%B4%20Gia%20T%E1%BB%B1&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
    "https://giaothong.hochiminhcity.gov.vn/expandcameraplayer/?camId=662b4de41afb9c00172d85c5&camLocation=H%E1%BA%A3i%20Th%C6%B0%E1%BB%A3ng%20L%C3%A3n%20%C3%94ng%20-%20Ch%C3%A2u%20V%C4%83n%20Li%C3%AAm&camMode=camera&videoUrl=https://d2zihajmogu5jn.cloudfront.net/bipbop-advanced/bipbop_16x9_variant.m3u8",
]

image_url_base = "https://giaothong.hochiminhcity.gov.vn/render/ImageHandler.ashx?id={}&t={}"
download_directory = "/content/images_caption"

os.makedirs(download_directory, exist_ok=True)

In [7]:
def get_camera_id_from_page_url(page_url):
    """Trích xuất camera ID từ URL trang."""
    parsed_url = urlparse(page_url)
    query_params = parse_qs(parsed_url.query)

    if 'camId' in query_params:
        return query_params['camId'][0]
    elif 'id' in query_params:
        return query_params['id'][0]
    return None

def process_camera(camera_id):
    """Tải ảnh cho một camera và gửi đến API Flask."""
    timestamp = int(time.time() * 1000)
    image_url = image_url_base.format(camera_id, timestamp)

    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    REQUEST_TIMEOUT_SECONDS = 3

    try:
        # --- Tải ảnh từ URL ảnh tĩnh với TIMEOUT và HEADERS ---
        print(f"  [Thread {camera_id}] Attempting to download image from {image_url}...")
        response = requests.get(image_url, stream=True, timeout=REQUEST_TIMEOUT_SECONDS, headers=HEADERS)
        response.raise_for_status()
        print(f"  [Thread {camera_id}] Successfully downloaded image.")

        filename = os.path.join(download_directory, f"{camera_id}.jpg")
        with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)

    except requests.exceptions.Timeout:
        print(f"  [Thread {camera_id}] TIMEOUT ERROR: Request timed out while downloading image from {image_url}")
    except requests.exceptions.RequestException as e:
        print(f"  [Thread {camera_id}] REQUEST ERROR: Could not download or send image: {e}")
    except Exception as e:
         print(f"  [Thread {camera_id}] AN UNEXPECTED ERROR occurred: {e}")

def caption_images_concurrently():
    camera_ids = []
    for url in camera_page_urls:
        camera_id = get_camera_id_from_page_url(url)
        if camera_id:
            camera_ids.append(camera_id)

    print(f"Found {len(camera_ids)} cameras. Starting concurrent processing...")


    MAX_WORKERS = 20

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Gửi từng camera_id đến hàm process_camera để chạy trong một luồng
        future_to_camera = {executor.submit(process_camera, cam_id): cam_id for cam_id in camera_ids}

        # Chờ tất cả các luồng hoàn thành và kiểm tra kết quả
        # As_completed trả về các Future khi chúng hoàn thành
        for future in concurrent.futures.as_completed(future_to_camera):
            cam_id = future_to_camera[future]
            try:
                # Lấy kết quả từ luồng (nếu hàm process_camera trả về gì đó)
                # future.result() sẽ ném ra ngoại lệ nếu có lỗi trong luồng
                future.result()
            except Exception as exc:
                print(f"  [Thread {cam_id}] generated an exception: {exc}")

    print("All camera processing tasks finished.")



In [8]:
def delete_all_files_in_folder(folder_path):

    try:
        # Lặp qua tất cả các mục trong thư mục
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            if os.path.isfile(file_path):
                # Xóa file
                os.remove(file_path)
                print(f"Đã xóa file: {file_path}")
        print(f"Đã xóa tất cả các file trong thư mục: {folder_path}")
    except FileNotFoundError:
        print(f"Lỗi: Thư mục '{folder_path}' không tồn tại.")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

# Modeling

In [9]:
import logging
import multiprocessing

In [10]:
vehicle_id_mapping = {
    0: "1",
    1: "2",
    2: "3",
    3: "4",
}

maximum_density = {
    "5b632a79fd4edb0019c7dc0f":100,
    "56de42f611f398ec0c481291":150,
    "56de42f611f398ec0c481293":150,
    "56de42f611f398ec0c481297":175,
    "5deb576d1dc17d7c5515ad20":200,
    "5b632b60fd4edb0019c7dc12":125,
    "5b728aafca0577001163ff7e":150,
    "5d8cd1f9766c880017188938":175,
    "5d8cd49f766c880017188944":175,
    "5deb576d1dc17d7c5515ad1f":270,
    "5deb576d1dc17d7c5515ad21":250,
    "63b3c274bfd3d90017e9ab93":100,
    "66b1c190779f740018673ed4":150,
    "66f126e8538c780017c9362f":125,
    "649da419a6068200171a6c90":125,
    "662b4de41afb9c00172d85c5":125,
    "662b4e8e1afb9c00172d865c":125,
    "662b4e201afb9c00172d85f9":200,
    "662b4ecb1afb9c00172d8692":150,
    "662b4efc1afb9c00172d86bc":150
    }

vehicle_name = {
    0: "Xe mô tô",
    1: "Xe ô tô",
    2: "Xe tải",
    3: "Xe bus (khách)",
}


#@app.route('/api/readings', methods=['POST'])
def predict_image(path_image):

    global model

    if multiprocessing.get_start_method(allow_none=True) != 'spawn':
          multiprocessing.set_start_method('spawn', force=True)
          logging.info("Multiprocessing start method set to 'spawn'.")
    else:
          logging.info("Multiprocessing start method is already 'spawn'.")
    # Kiểm tra xem model đã được tải chưa
    try:
      count = []
      result = model.predict(path_image)

      boxes = result[0].boxes.xyxy.int().cpu().tolist()
      classes = result[0].boxes.cls.int().cpu().tolist()


      for box, cls_index in zip(boxes, classes):
          class_id = int(cls_index)

      class_counts = collections.Counter(classes)

      sorted_class_ids = sorted(vehicle_id_mapping.keys())
      summary_parts = []

      sorted_class_ids = sorted(vehicle_id_mapping.keys()) # Sắp xếp theo thứ tự ID
      for class_id in sorted_class_ids:
          # Lấy tên hiển thị cho lớp (từ result.names)
          display_name = vehicle_name.get(class_id, f"Class {class_id}")

          counted = class_counts.get(class_id, 0)

          # Định dạng chuỗi cho từng loại: "tên {id_gốc}: số lượng"
          summary_parts.append(f"{display_name}: {counted}")

      # Nối các phần lại bằng dấu chấm phẩy và khoảng trắng
      summary_text = " ; ".join(summary_parts)

      for class_id in sorted_class_ids:
          temp = class_counts.get(class_id, 0)
          count.append(temp)

      density = count[0] + count[1]*5+ count[2]*15 + count[3]*20

      camera_id = path_image.split("/")[-1].split(".")[0]
      maximum_score_density = maximum_density.get(camera_id,0)



      camera_ref = db.collection('cameras').document(camera_id)
      timestamp_ms = int(time.time() * 1000)

      if density >= maximum_score_density:
        density = 1.0
      else:
        density =  float(density/maximum_score_density)

      doc = camera_ref.get()
      data_to_save = {
        'density': density,
        'timestamp': timestamp_ms,
        'summary': summary_text,
        'status': 'OK',
      }

      camera_ref.set(data_to_save, merge=True)
      print("Done!")

    except Exception as e:
        print(f"ERROR processing image or saving to Firestore: {e}")
        return jsonify({"msg": f"Error processing image or saving to Firestore: {e}"}), 500






In [11]:



def predict_images_concurrently():





    images_caption = "/content/images_caption"
    path_list = []
    for image in os.listdir(images_caption):
          path_image = os.path.join(images_caption, image)
          path_list.append(path_image)

    print(f"Found {len(path_list)} cameras. Starting concurrent processing...")


    MAX_WORKERS = 10

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Gửi từng camera_id đến hàm process_camera để chạy trong một luồng
        future_to_path = {executor.submit(predict_image, path_image): path_image for path_image in path_list}

        # Chờ tất cả các luồng hoàn thành và kiểm tra kết quả
        # As_completed trả về các Future khi chúng hoàn thành
        for future in concurrent.futures.as_completed(future_to_path):
            path_image = future_to_path[future]
            try:
                # Lấy kết quả từ luồng (nếu hàm process_camera trả về gì đó)
                # future.result() sẽ ném ra ngoại lệ nếu có lỗi trong luồng
                future.result()
            except Exception as exc:
                print(f"  [Thread {path_image}] generated an exception: {exc}")

    print("All camera processing tasks finished.")

# POST

In [ ]:
if __name__ == '__main__':
    initialize_firebase()
    load_model()
    while True:
        delete_all_files_in_folder(download_directory)
        caption_images_concurrently()
        predict_images_concurrently()

        WAIT_INTERVAL_SECONDS = 10

        time.sleep(WAIT_INTERVAL_SECONDS)

    else:
        print("Failed to load model. Exiting.")


Firestore client obtained.
Model PyTorch loaded successfully from /content/drive/MyDrive/Project/Dự án cntt 2 /dataset/YOLO8m_v1_0001.pt
Đã xóa file: /content/images_caption/662b4efc1afb9c00172d86bc.jpg
Đã xóa file: /content/images_caption/66b1c190779f740018673ed4.jpg
Đã xóa file: /content/images_caption/5d8cd49f766c880017188944.jpg
Đã xóa file: /content/images_caption/56de42f611f398ec0c481297.jpg
Đã xóa file: /content/images_caption/662b4e201afb9c00172d85f9.jpg
Đã xóa file: /content/images_caption/5deb576d1dc17d7c5515ad20.jpg
Đã xóa file: /content/images_caption/56de42f611f398ec0c481293.jpg
Đã xóa file: /content/images_caption/662b4e8e1afb9c00172d865c.jpg
Đã xóa file: /content/images_caption/5b632b60fd4edb0019c7dc12.jpg
Đã xóa file: /content/images_caption/56de42f611f398ec0c481291.jpg
Đã xóa file: /content/images_caption/662b4ecb1afb9c00172d8692.jpg
Đã xóa file: /content/images_caption/5b728aafca0577001163ff7e.jpg
Đã xóa file: /content/images_caption/66f126e8538c780017c9362f.jpg
Đã